In [2]:
import numpy as np
import plotly.express as px
import math
import scipy.stats as st
import plotly.figure_factory as ff

$λ$: Taxa de chegada

$μ$: Taxa de atendimento / velocidade do servidor

$ρ$: Percentual de tempo que o sistema permanece ocupado Marcos

$w$: Tempo médio de permanência do cliente no sistema (Fila + Atendimento) OK

$w_q$:Tempo médio de permanência do cliente na fila OK

$L$: Média de clientes no sistema  

$L_q$: Média de clientes na fila

$π_n$: Probabilidade de haver n clientes na fila. 

In [11]:
def simulador(simulation_total_time, lambda_, mi_):
    n = 0
    simul_time = waiting_line_time = number_of_arrivals = system_spent_time = clients_in_system = clients_in_line = last_event_time = 0
    line_queue = []
    arrivals = []

    while simul_time < simulation_total_time:
        arrival_time = np.random.exponential(scale=1/lambda_)
        departure_time = np.random.exponential(scale=1/mi_)
        last_event_time = simul_time

        if n == 0 or arrival_time < departure_time:
            simul_time += arrival_time
            number_of_arrivals += 1
            clients_in_system += (simul_time - last_event_time) * n
            clients_in_line += (simul_time - last_event_time) * len(line_queue)
            n += 1
            arrivals.append(simul_time)

            if n > 1:
                line_queue.append(simul_time)

        else:
            simul_time += departure_time
            clients_in_system += (simul_time - last_event_time) * n
            clients_in_line += (simul_time - last_event_time) * len(line_queue)

            if len(line_queue) > 0:
                waiting_line_time += simul_time - line_queue.pop(0)
            n -= 1
            system_spent_time += simul_time - arrivals.pop(0)

    avg_waiting_time = waiting_line_time / number_of_arrivals
    avg_system_time = system_spent_time / number_of_arrivals
    avg_clients_in_system = clients_in_system / simul_time
    avg_clients_in_line = clients_in_line / simul_time

    return n, simul_time, avg_waiting_time, avg_system_time, number_of_arrivals, avg_clients_in_system, avg_clients_in_line

In [21]:
total_simulation_time = 100000
_lambda = 1
_mi = 4

In [22]:
n, simul_time, mean_waiting_inline_time, mean_system_spent_time, number_of_arrivals, mean_clients_in_system, mean_clients_in_line = simulador(total_simulation_time, _lambda, _mi)

In [23]:
print(f"\nTotal de pessoas que foram ao banco na simulação: {number_of_arrivals}")
print(f"Número de pessoas na fila ao final da simulação: {n}")
print(f"Tempo total da simulação: {simul_time:.4f}")
print(f"Tempo médio do cliente no sistema: {(mean_system_spent_time):.4f}")
print(f"Tempo médio de espera na fila: {(mean_waiting_inline_time):.4f}")
print(f"Número médio de clientes no sistema: {(mean_clients_in_system):.4f}")
print(f"Número médio de clientes na fila: {(mean_clients_in_line):.4f}")


Total de pessoas que foram ao banco na simulação: 99688
Número de pessoas na fila ao final da simulação: 1
Tempo total da simulação: 100000.4030
Tempo médio do cliente no sistema: 0.3333
Tempo médio de espera na fila: 0.0829
Número médio de clientes no sistema: 0.3323
Número médio de clientes na fila: 0.0826


In [24]:
def simulator_confidence_interval(sample_size, conf_level):
    
    results = [simulador(1500, 1, 2)[2] for _ in range(sample_size)]

    proportion = np.sum(results) / sample_size
    std_error = math.sqrt(proportion * (1 - proportion) / sample_size)
    margin_error = std_error * st.norm.ppf(conf_level)
    lower_bound = proportion - margin_error
    upper_bound = proportion + margin_error

    return (lower_bound, upper_bound), results

In [25]:
intervalo, data = simulator_confidence_interval(1000, 0.95)
print(intervalo)

(0.4692551486582801, 0.5212676514767427)


In [26]:
px.box(data, orientation='h')

In [27]:
hist_data = [data]
group_labels = ['']

fig = ff.create_distplot(hist_data, group_labels, curve_type='normal', bin_size=0.009)
fig.show()

## Avaliando Resultados

Link para referência: https://people.revoledu.com/kardi/tutorial/Queuing/MM1-Queuing-System.html

$W$ = tempo médio que o cliente permanece no sistema (tempo de espera na fila + tempo de serviço). ${W = \frac{1}{\mu - \lambda}}$.\
$W_q$ = tempo médio que os clientes devem esperar na fila até serem atendidos. ${W_q = W - \frac{1}{\mu}}$.\
$L$ = número médio de clientes no sistema (na fila e sendo servidos). ${L = \lambda W}$\
$L_q$ = número médio de clientes na fila. ${L_q = \lambda W_q}$

### Caso ${\lambda} = 1$ e ${\mu} = 2$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``99388``\
Número de pessoas na fila ao final da simulação: ``1``\
Tempo total da simulação: ``100000.6943``\
Tempo médio do cliente no sistema: ``1.0001``\
Tempo médio de espera na fila: ``0.4995``\
Número médio de clientes no sistema: ``0.9940``\
Número médio de clientes na fila: ``0.4965``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 1}$ e ${\mu = 2}$ temos: ${W = \frac{1}{2 - 1}}$ = 1\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= 1}$ e ${\mu = 2}$ temos: ${W_q = 1 - \frac{1}{2}}$ = ${\frac{1}{2}}$\
${L = \lambda W}$. Logo, com ${W= 1}$ e ${\lambda = 1}$ temos: ${L = 1}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{2}}$ e ${\lambda = 1}$ temos: ${L_q = \frac{1}{2}}$

Os resultados condizem com os valores da simulação!!

### Caso ${\lambda} = 2$ e ${\mu} = 4$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``200215``\
Número de pessoas na fila ao final da simulação: ``8``\
Tempo total da simulação: ``100000.0323``\
Tempo médio do cliente no sistema: ``0.5056``\
Tempo médio de espera na fila: ``0.2556``\
Número médio de clientes no sistema: ``1.0124``\
Número médio de clientes na fila: ``0.5117``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 2}$ e ${\mu = 4}$ temos: ${W = \frac{1}{4 - 2}} = \frac{1}{2}$\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= \frac{1}{2}}$ e ${\mu = 4}$ temos: ${W_q = \frac{1}{2} - \frac{1}{4}}$ = ${\frac{1}{4}}$\
${L = \lambda W}$. Logo, com ${W= \frac{1}{2}}$ e ${\lambda = 2}$ temos: ${L = 2\frac{1}{2} = 1}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{4}}$ e ${\lambda = 2}$ temos: ${L_q = \frac{2}{4} = \frac{1}{2}}$

Os resultados condizem com os valores da simulação!!

### Caso ${\lambda} = 1$ e ${\mu} = 4$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``99688``\
Número de pessoas na fila ao final da simulação: ``1``\
Tempo total da simulação: ``100000.4030``\
Tempo médio do cliente no sistema: ``0.3333``\
Tempo médio de espera na fila: ``0.0829``\
Número médio de clientes no sistema: ``0.3323``\
Número médio de clientes na fila: ``0.0826``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 1}$ e ${\mu = 4}$ temos: ${W = \frac{1}{4 - 1}} = \frac{1}{3}$\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= \frac{1}{3}}$ e ${\mu = 4}$ temos: ${W_q = \frac{1}{3} - \frac{1}{4}}$ = ${\frac{1}{12}}$\
${L = \lambda W}$. Logo, com ${W= \frac{1}{3}}$ e ${\lambda = 1}$ temos: ${L = \frac{1}{3}}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{12}}$ e ${\lambda = 1}$ temos: ${L_q = \frac{1}{12}}$

Os resultados condizem com os valores da simulação!!